In [79]:
## TODO: improve the regex detection and other shit

In [178]:
import re
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [179]:
df = pd.read_csv('processed.csv')

In [180]:
df.head()

,Unnamed: 0,entity_name,entity_value,image_data,group_id
0,0,weight,500.0 gram,[],748919
1,1,volume,1.0 cup,[],916768
2,2,weight,0.709 gram,"[{'word': '02g', 'metadata': {'word': '02g', '...",459516
3,3,weight,0.709 gram,"[{'word': '2g', 'metadata': {'word': '2g', 'po...",459516
4,4,weight,1400 milligram,[],731432


In [181]:
# Unit normalization mapping
entity_unit_map = {
    'weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

def normalize_unit(word):
    # Convert units to a standard form
    unit_patterns = {
        r'g|gram|grams': 'gram',
        r'kg|kilogram|kilograms': 'kilogram',
        r'cm|centimetre|centimetres': 'centimetre',
        r'mm|millimetre|millimetres': 'millimetre',
        r'inch|inches': 'inch',
        r'foot|feet|ft': 'foot',
        r'lb|lbs|pound|pounds': 'pound'
        # Add more mappings if necessary
    }
    for pattern, unit in unit_patterns.items():
        if re.search(pattern, word, re.IGNORECASE):
            return unit
    return word

In [182]:
def extract_number_and_unit(word):
    patterns = {
        'weight': r'(\d+(\.\d+)?)\s?(g|gram|grams|kg|kilogram|kilograms|lb|lbs|pound|pounds)',
        'length': r'(\d+(\.\d+)?)\s?(cm|centimetre|centimetres|mm|millimetre|millimetres|inch|feet|ft)',
        'voltage': r'(\d+(\.\d+)?)\s?(V|volts|kV|kilovolts)',
        'wattage': r'(\d+(\.\d+)?)\s?(W|watts|kW|kilowatts)',
        'volume': r'(\d+(\.\d+)?)\s?(L|litres|ml|millilitres|gallon|cup|pint|quart)'
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, word, re.IGNORECASE)
        if match:
            value = float(match.group(1))
            unit = normalize_unit(match.group(3))
            return value, unit
    return None, None

In [183]:
import ast

# Function to parse the string and extract features
def extract_data_from_image(image_data_str):
    # Parse the string to convert it to a Python list of dictionaries
    try:
        image_data = ast.literal_eval(image_data_str)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing image_data: {e}")
        return []

    # Ensure image_data is a list
    if isinstance(image_data, list):
        features = []
        for entry in image_data:
            # Each entry should be a dictionary, access 'metadata' and 'position' fields
            metadata = entry.get('metadata', {})
            position = metadata.get('position', {})

            # Extract the relevant fields with default values in case of missing data
            x = position.get('x', 0)
            y = position.get('y', 0)
            width = position.get('width', 0)
            height = position.get('height', 0)
            confidence = metadata.get('confidence', 0)
            word = entry.get('word', '')
            orientation = entry.get('orientation', '0 degrees')

            # Append the extracted features as a dictionary
            features.append({
                'x': x,
                'y': y,
                'width': width,
                'height': height,
                'confidence': confidence,
                'word': word,
                'orientation': orientation
            })

        return features
    else:
        # If image_data is not a list, return an empty list or raise an error
        return []

In [184]:
df['extracted_image_data'] = df['image_data'].apply(extract_data_from_image)

In [185]:
df = df.drop(columns=['Unnamed: 0'])

In [186]:
df.head()

,entity_name,entity_value,image_data,group_id,extracted_image_data
0,weight,500.0 gram,[],748919,[]
1,volume,1.0 cup,[],916768,[]
2,weight,0.709 gram,"[{'word': '02g', 'metadata': {'word': '02g', '...",459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2..."
3,weight,0.709 gram,"[{'word': '2g', 'metadata': {'word': '2g', 'po...",459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1..."
4,weight,1400 milligram,[],731432,[]


In [187]:
df = df.drop(columns = ['image_data'])

In [188]:
df.head()

,entity_name,entity_value,group_id,extracted_image_data
0,weight,500.0 gram,748919,[]
1,volume,1.0 cup,916768,[]
2,weight,0.709 gram,459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2..."
3,weight,0.709 gram,459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1..."
4,weight,1400 milligram,731432,[]


In [189]:
## extraction of actual text

unit_conversion = {
    'g': 1, 'gram': 1, 'grams': 1,
    'kg': 1000, 'kilogram': 1000, 'kilograms': 1000,
    'mg': 0.001, 'milligram': 0.001,
    'cm': 1, 'centimeter': 1, 'centimeters': 1,
    'm': 100, 'meter': 100, 'meters': 100,
    'inch': 2.54, 'foot': 30.48
}

def convert_to_common_unit(value, unit):
    unit = unit.lower()
    if unit in unit_conversion:
        return value * unit_conversion[unit]
    return value

In [190]:
import re

def extract_number_and_unit(text):
    # Regular expression to extract number and unit
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', text)
    if match:
        number = float(match.group(1))
        unit = match.group(3).lower()
        return number, unit
    return None, None

def normalize_value(text):
    # Normalize values by extracting number and unit
    number, unit = extract_number_and_unit(text)
    if number is not None and unit is not None:
        return convert_to_common_unit(number, unit)
    return None

In [191]:
def match_entity_value(row):
    # Extract number and unit from entity_value
    entity_value = row['entity_value']
    entity_number, entity_unit = extract_number_and_unit(entity_value)
    
    if entity_number is not None and entity_unit is not None:
        entity_value_normalized = convert_to_common_unit(entity_number, entity_unit)
        
        matched = []
        for item in row['extracted_image_data']:
            word = item['word']
            image_number = normalize_value(word)
            if image_number is not None and image_number == entity_value_normalized:
                matched.append(1)
            else:
                matched.append(0)
        return matched
    else:
        return [0] * len(row['extracted_image_data'])

In [192]:
df['labels'] = df.apply(match_entity_value, axis=1)

In [193]:
df.head()

,entity_name,entity_value,group_id,extracted_image_data,labels
0,weight,500.0 gram,748919,[],[]
1,volume,1.0 cup,916768,[],[]
2,weight,0.709 gram,459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2...","[0, 0, 0, 0, 0]"
3,weight,0.709 gram,459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,weight,1400 milligram,731432,[],[]


In [194]:
# Prepare features like x, y, width, height, orientation, confidence
def extract_features(image_data):
    features = []
    for item in image_data:
        features.append([item['x'], item['y'], item['width'], item['height'], item['orientation'], item['confidence']])
    return np.array(features).flatten()

df['features'] = df['extracted_image_data'].apply(extract_features)

In [195]:
df.head()

,entity_name,entity_value,group_id,extracted_image_data,labels,features
0,weight,500.0 gram,748919,[],[],[]
1,volume,1.0 cup,916768,[],[],[]
2,weight,0.709 gram,459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2...","[0, 0, 0, 0, 0]","[723, 698, 34, 22, 0 degrees, 69, 289, 328, 17..."
3,weight,0.709 gram,459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[457, 713, 30, 10, 0 degrees, 12, 422, 827, 8,..."
4,weight,1400 milligram,731432,[],[],[]


In [196]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [197]:
def prepare_features_and_labels(df):
    expanded_features = []
    expanded_labels = []

    for idx, row in df.iterrows():
        features = row['features']
        labels = row['labels']
        group_id = row['group_id']
        entity_name = row['entity_name']
        
        
        # Split features and labels

        ## TODO : this is bad do not hardcode here, make a global constant
        num_features_per_sample = 6
        num_samples = len(labels)

        for i in range(num_samples):
            feature_set = features[i*num_features_per_sample:(i+1)*num_features_per_sample]
            label = labels[i]
            expanded_features.append(feature_set)
            expanded_labels.append(label)
    
    return pd.DataFrame(expanded_features, columns=[f'feature_{i}' for i in range(len(expanded_features[0]))]), np.array(expanded_labels)

# Prepare data
features_df, labels_array = prepare_features_and_labels(df)


In [198]:
features_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5
0,723,698,34,22,0 degrees,69
1,289,328,17,16,0 degrees,57
2,499,753,17,25,0 degrees,0
3,439,651,17,17,0 degrees,29
4,361,723,22,34,0 degrees,69


In [173]:
final_features = pd.concat([features_df, pd.DataFrame(df['group_id']), pd.DataFrame(df['entity_name'])], axis=1)
final_features.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,group_id,entity_name
0,723,698,34,22,0 degrees,69,748919.0,weight
1,289,328,17,16,0 degrees,57,916768.0,volume
2,499,753,17,25,0 degrees,0,459516.0,weight
3,439,651,17,17,0 degrees,29,459516.0,weight
4,361,723,22,34,0 degrees,69,731432.0,weight


In [200]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(final_features, final_labels, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

ValueError: could not convert string to float: np.str_('0 degrees')